<a href="https://colab.research.google.com/github/EviEno/Dog-Breed-Prediction-with-Flask-app/blob/main/Dog_Breed_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dog Breed Prediction

## This project we will see how to use Keras and Tensorflow to build, train, and test s Convolution Neural Network capable of identifying the breed dog in supplied image. This is a supervised learning problem, specifically a multiclass classification problem.

In [2]:
#Run this cell and select the kaggle.json file download
#from the Kaggle account setting page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"evinovitasari","key":"7c8b9577e1faf78aa0c9bf9881aff842"}'}

### We will start by connection to Kaggle using Kaggle API which can be download from your Kaggle's account setting and uploading it here(Upload box)

In [4]:
# Next, install the Kaggle API client
!pip install -q kaggle

### Next we will install Kaggle API using pip instalation

In [5]:
# The Kaggle API client expects this file to be in ~/.kaggle, so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup
!chmod 600 ~/.kaggle/kaggle.json

### Setting up Kaggle API

In [6]:
# Creating directory and changing the current working directory
!mkdir dog_dataset
%cd dog_dataset

/content/dog_dataset


### To store the data will create a new directory and make it as current working directory.

In [7]:
# Serching for dataset
!kaggle dataset list -s dogbreedidfromcomp

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...
kaggle: error: argument command: invalid choice: 'dataset' (choose from 'competitions', 'c', 'datasets', 'd', 'kernels', 'k', 'models', 'm', 'files', 'f', 'config')


### Searching Kaggle for the required dataset using search option(-s) with title 'dogbreedid fromcomp'. We can also used different search options like searching competitions, notebooks, kernels. dataset, etc.

In [8]:
# Downloading dataset and coming out of directory
!kaggle dataset download catherinehorng/dogbreedidfromcomp
%cd ..

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...
kaggle: error: argument command: invalid choice: 'dataset' (choose from 'competitions', 'c', 'datasets', 'd', 'kernels', 'k', 'models', 'm', 'files', 'f', 'config')
/content


After searching the data next step would be downloading the data into collab notebook using references found in search option.

In [9]:
# Unzipping downloaded file and removing unusable file
!unzip dog_daraset/dogbreedidfromcomp.zip -d dog_dataset
!rm dog_dataset/dogbreedidfromcomp.zip
!rm dog_dataset/samle_submission.csv


unzip:  cannot find or open dog_daraset/dogbreedidfromcomp.zip, dog_daraset/dogbreedidfromcomp.zip.zip or dog_daraset/dogbreedidfromcomp.zip.ZIP.
rm: cannot remove 'dog_dataset/dogbreedidfromcomp.zip': No such file or directory
rm: cannot remove 'dog_dataset/samle_submission.csv': No such file or directory


###We will unzip the data which is donwloaded and remove the irrelevant fikes

In [18]:
# Important library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam

### Importing Required libraries.

In [ ]:
# Read the labels.cvs file and checking shape and records
labels_all = pd.read_csv("dog_dataset/labels.csv")
print(labels_all.shape)
labels_all.head()

###Loading the labels data into dataframe and viewing it. Here we analyses that labels contains 10222 rows and 2 columns

In [ ]:
# Visualize the number of each breeds
breeds_all = labels_all["breed"]
breed_counts = breeds_all.value_counts()
breed_counts.head()

Here we are finding out the count per class i.e. total data in each class using value_counts() function

In [ ]:
## Selecting first 3 breeds (Limitation due to computation power)
CLASS_NAMES = ['scottish_deerhound','maltese_dog','bernese_mountain_dog']
labels = labels_all[(labels_all['breed'].isin(CLASS_NAMES))]
labels = labels.reset_index()
labels.head()

### We will on only 3 breed due to limited computational power. You can consider more classes as per your system computation power.

In [ ]:
# Creating numpy matrix with zeros
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
# One hot encoding
Y_data = label_binarize(labels['breed'], classes = CLASS_NAMES)

# Reading and converting image to numpy array and normalizing dataset
for i in tqdm(range(len(labels))):
    img = image.load_img('dog_dataset/train/%s.jpg' % labels['id'][i], target_size=(224, 224))
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x / 255.0

# Printing train image and one hot encode shape & size
print('\nTrain Images shape: ',X_data.shape,' size: {:,}'.format(X_data.size))
print('One-hot encoded output shape: ',Y_data.shape,' size: {:,}'.format(Y_data.size))

### As we are working the classification dataset firts we need to one hot encoded the target value i.e. the class. After that we will read images and convert them into numoy array and finally normalize the array

In [ ]:
# Building the Model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 16, kernel_size = (7,7), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8, kernel_size = (5,5), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(64, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(len(CLASS_NAMES), activation = "softmax"))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

model.summary()

### Next we will create a network architecture for the model. We have used different types of layers according to their features namely Conv_2d (It is used to create a convolutional kernel that is convolved with the input layer to produce the output tensor), max_pooling2d (It is a downsampling technique which takes out the maximum value over the window defined by poolsize), flatten (It flattens the input and creates a 1D output), Dense (Dense layer produce the output as the dot product of input and kernel).

After defining the network architecture we found out the total parameters as 162,619.


In [ ]:
# Splitting the data set into training and testing data sets
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_data, Y_data, test_size = 0.1)
# Splitting the training data set into training and validation data sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)

After defining the network architecture we will start with splitting the test and train data then diving train data in train and validation

In [ ]:
# Training the model
epochs = 100
batch_size = 128

history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs,
                    validation_data = (X_val, Y_val))

### Now we will train our models on 100 epochs and batch size 128. You can try using more number of epochs to increases accuracy. During each epochs we can see how the model is performing by viewing the training and validation accuracy

In [ ]:
# Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])

plt.show()

### Here we analyses how the model is learning with each epoch interm of accuracy

In [ ]:
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

### We will uses predict function to make predictions using this model also we are finding out the accuracy on the tet set.

In [ ]:
# Plotting image to compare
plt.imshow(X_test[1,:,:,:])
plt.show()

# Finding max value from predition list and comaparing original value vs predicted
print("Originally : ",labels['breed'][np.argmax(Y_test[1])])
print("Predicted : ",labels['breed'][np.argmax(Y_pred[1])])

### Here you can see image with